In [ ]:
%pip install dask
%pip install dask[dataframe]
%pip install dask[complete]
%pip install dask[distributed]
%pip install cloudpickle
%pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install transformers
%pip install pytorch_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.7 MB/s 
     |████████████████████████████████| 163 kB 88.4 MB/s 
     |████████████████████████████████| 7.6 MB 71.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 4.8 MB/s 
     |████████████████████████████████| 132 kB 49.0 MB/s 
     |████████████████████████████████| 1.3 MB 87.0 MB/s 
     |████████████████████████████████| 880 kB 78.6 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 9.2 MB 73.5 MB/s 
     |████████████████████████████████| 140 kB 86.0 MB/s 
     |████████████████████████████████| 127 kB 82.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=342aefbc57db701a4788fb890a2ccef24c27363160931d313b8ad72ce2528188
  Stored

In [ ]:
# define the imports
try:
  import os
  import json
  import math
  import dask
  from dask.distributed import Client
  import dask.dataframe as dd
  from dask.diagnostics import ProgressBar
  import dask.multiprocessing
  import numpy as np
  import pandas as pd
  import torch
  from transformers import BertTokenizer
  from transformers import BertModel
  from tqdm.notebook import tqdm
  tqdm.pandas()
  from tqdm import tqdm
except Exception as ex:
  print('Some modules are missing : {}'.format(ex))

Some modules are missing : No module named 'transformers'


In [ ]:
dtypes = {
  'id': str,
  'news': str    
}

In [ ]:
%%time
base_uri = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/'
data_uri = '{base_uri}news_with_header.csv'.format(base_uri=base_uri)
df = pd.read_csv(data_uri, dtype=dtypes)

CPU times: user 9.74 s, sys: 771 ms, total: 10.5 s
Wall time: 13 s


In [ ]:
%%time
## Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states=True)

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 11.8 s, sys: 1.95 s, total: 13.8 s
Wall time: 15.1 s


In [ ]:
%%time
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

CPU times: user 974 µs, sys: 0 ns, total: 974 µs
Wall time: 982 µs


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [ ]:
def get_vectors(row):
  encoding = tokenizer.encode(row['news'], add_special_tokens=True,
                              max_length=512, truncation=True, padding="max_length")
  token_text = tokenizer.convert_ids_to_tokens(encoding)
  indexed_tokens = tokenizer.convert_tokens_to_ids(token_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])

  # Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
  with torch.no_grad():
    outputs = model(tokens_tensor)

    # Evaluating the model will return a different number of objects based on how it's  configured in the `from_pretrained` call earlier.
    # In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers.
    # See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

    # initial embeddings can be taken from 0th layer of hidden states
    word_embed = hidden_states[0]

    output_path = '{base_uri}/vectors/{id}.npy'.format(base_uri=base_uri, id=row['id'])
    np.save(output_path, word_embed)
    return word_embed

In [ ]:
# TEST
# _df = df.sample(20)
# len(_df)

In [ ]:
%%time
parallel_tasks = []
for idx, row in tqdm(df.iterrows()):
  task = dask.delayed(get_vectors)(row)
  parallel_tasks.append(task)

1727888it [04:21, 6604.68it/s]

CPU times: user 4min 18s, sys: 28.5 s, total: 4min 47s
Wall time: 4min 21s


In [ ]:
len(parallel_tasks)

1727888

In [ ]:
%%time
with ProgressBar():
  result = dask.compute(*parallel_tasks)
print('DONE')  

[#                                       ] | 2% Completed |  2hr 55min  1.1s